In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re

In [120]:
xls = pd.ExcelFile("Product Matching Dataset.xlsx")

In [121]:
master_df = pd.read_excel(xls, "Master File")
dataset_df = pd.read_excel(xls, "Dataset")

In [122]:
dataset_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [123]:
master_df.head()

,sku,product_name,product_name_ar,price
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0


In [124]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83562 entries, 0 to 83561
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sku                          83562 non-null  int64  
 1   marketplace_product_name_ar  83562 non-null  object 
 2   seller_item_name             83562 non-null  object 
 3   price                        83562 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.6+ MB


## 1. Data Preprocessing

### a. Cleaning

In [125]:
# remove punctuation and convert to lowercase
dataset_df['seller_item_name'] = dataset_df['seller_item_name'].map(lambda s: re.sub(r'[^\w\s]','',s.lower()))

#### Generating Dictionary from master file

In [126]:
master_df.duplicated().sum()

0

In [127]:
master_df.isna().sum()

sku                0
product_name       0
product_name_ar    0
price              0
dtype: int64

In [161]:
def normalize_english(text):
    text = str(text).lower()
    # remove numbers (for master file only)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) # remove non-alphanumeric characters
    # text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = text.strip()
    return text


def normalize_arabic(text):
    text = str(text).lower()
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove Arabic diacritics (tashkeel)
    # text = re.sub(r'[^\u0621-\u064A\s]', '', text) # remove non-arabic characters
    text = re.sub(r'[^\u0621-\u064A0-9\s]', ' ', text) # remove non-arabic characters except for numbers
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces

    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه ', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r"(\d+)", r" \1 ", text)
    text = re.sub(r"\b[\u0600-\u06FF]\b", "", text)
    text = re.sub(r'([\u0600-\u06FF])\1{2,}', r'\1\1', text)
    text = re.sub(r'([\u0600-\u06FF])\1+', r'\1', text) # remove arabic repetition
    # Remove short/incomplete words (single characters or partial words)
    text = re.sub(r'\b\w\b', '', text)  # Removes standalone single characters
    text = re.sub(r'\b(?:سعر جديد|س جديد|س جدي|س ج|ركز)\b', '', text)# Remove specific unwanted phrases
    text = re.sub( r'مرهم|اكريم', 'كريم', text)
    text = re.sub( r'قرص|اقراص|كبسوله', 'كبسول', text)
    text = re.sub( r'اقراص|شريط|شرائط|شريطين', 'قرص', text)
    text = re.sub( r'امبولات|امبوله|حقن', 'امبول', text)
    text = re.sub( r'لبوس|لبوس اطفال|لبوس اطفال فاركو', 'اقماع للاطفال', text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [162]:
normalize_english(" we.ork//  / Hello,World!  ")

'we ork hello world'

In [163]:
normalize_arabic("helloo فيدروب 2800 وحدة دولية//مل نقط _ بالفم helloo 15 مل")

'فيدروب 2800 وحده دوليه مل نقط بالفم 15 مل'

In [164]:
master_df['product_name_clean'] = master_df['product_name'].map(normalize_english)
master_df['product_name_ar_clean'] = master_df['product_name_ar'].map(normalize_arabic)

In [165]:
master_df.head()

,sku,product_name,product_name_ar,price,product_name_clean,product_name_ar_clean
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0,anafronil mg tab,انافرونيل 75 مجم اس ار 20 كبسول
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5,loprecough syrup ml,لوبريكاف شراب 100 مل
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0,tomex plus tab,تومكس بلس 50 كبسول
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0,tarolimus oint gm,تاروليمس 03 كريم 15 جم
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0,gliptus plus mg tab,جليبتس بلس 50 1000 مجم 30 كبسول


In [166]:
from collections import Counter

english_words = Counter(" ".join(master_df['product_name_clean']).split())
arabic_words = Counter(" ".join(master_df['product_name_ar_clean']).split())

# filter out one-letter words
english_words = {k:v for k,v in english_words.items() if len(k) > 2}
arabic_words = {k:v for k,v in arabic_words.items() if len(k) > 2}

english_words = Counter(english_words)
arabic_words = Counter(arabic_words)

In [167]:
english_words

Counter({'tab': 509,
         'cap': 129,
         'syrup': 44,
         'plus': 38,
         'amp': 37,
         'eye': 37,
         'cream': 36,
         'drops': 35,
         'susp': 27,
         'sachets': 18,
         'mcg': 16,
         'milk': 16,
         'spray': 15,
         'nasal': 14,
         'drop': 13,
         'vial': 12,
         'oint': 10,
         'oral': 10,
         'dose': 10,
         'erastapex': 9,
         'baby': 9,
         'solution': 8,
         'gel': 8,
         'adult': 8,
         'with': 8,
         'advance': 8,
         'limitless': 8,
         'panadol': 6,
         'vaginal': 6,
         'empacoza': 6,
         'lozenges': 6,
         'syringe': 6,
         'hero': 6,
         'tareg': 6,
         'pre': 6,
         'filled': 6,
         'voltaren': 6,
         'supp': 5,
         'extra': 5,
         'conventin': 5,
         'sachet': 5,
         'amaryl': 5,
         'tritace': 5,
         'retard': 5,
         'dolcyl': 5,
         'vit': 5,


In [168]:
def save_symspell_dict(words_dict, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for word, freq in words_dict.items():
            f.write(f"{word} 1000000\n")

In [169]:
save_symspell_dict(english_words, "symspell_english.txt")
save_symspell_dict(arabic_words, "symspell_arabic.txt")

In [170]:
from symspellpy import SymSpell, Verbosity

sym_spell_en = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell_ar = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell_en.load_dictionary("symspell_english.txt", term_index=0, count_index=1)
sym_spell_ar.load_dictionary("symspell_arabic.txt", term_index=0, count_index=1, encoding="utf-8")

True

In [171]:
def correct_spelling(sentence):
    # words = sentence.split()  # Split sentence into words
    # corrected_words = []

    # for word in words:
    #     suggestions = sym_spell_ar.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    #     corrected_word = suggestions[0].term if suggestions else word  # Use first suggestion
    #     corrected_words.append(corrected_word)

    # return " ".join(corrected_words)  # Recombine corrected words

    suggestion = sym_spell_ar.lookup_compound(sentence, max_edit_distance=2)
    if suggestion:
        return suggestion[0].term
    return sentence

In [173]:
correct_spelling("كسبول")

'كبسول'

In [149]:
correct_spelling("تيلفاستت")

'تلفاست'

In [150]:
correct_spelling("كلوبترا")

'كلوباترا'

In [174]:
correct_spelling("كلوبترا جابتين دانسيت")

'كلوباترا جابتن دانست'

In [157]:
correct_spelling("جابتين")

'جابتن'

In [175]:
correct_spelling("جدييييد")

'جديد اسيد'

In [179]:
dataset_df['seller_item_name_clean'] = dataset_df['seller_item_name'].map(lambda x : normalize_arabic(str(x)))

In [183]:
dataset_df['seller_item_name_clean'] = dataset_df['seller_item_name'].map(lambda x: correct_spelling(str(x)))

In [180]:
normalize_arabic("تورسيرتيك 5مجم")

'تورسيرتيك مجم'

In [184]:
dataset_df.sample(10)

,sku,marketplace_product_name_ar,seller_item_name,price,seller_item_name_clean
12688,370,فوربيودس 400/12 مكجم 60 كبسول استنشاق,فوربيدوس400مجم60 كبسول,334.0,فوربيدوس400مجم60 كبسول
10195,557,زيرتيك 0.1% شراب 100 مل,زيرتك شراب سعر جدبد,32.5,زيرتك شراب لشعر جديد
64192,925,جليبتس 50 مجم 30 قرص,جليبتس50مجم عادى30قرص,151.5,جليبتس مجم عادى30قرص
65708,1534,رومانتيجرا 20 مجم 4 قرص,رومانتيجرا اقراص,37.5,رومانتيجرا قرص
40258,1771,ابيمول 500 مجم 20 قرص,ابيمول اقراص جلاكسو,13.0,ابيمول قرص مجم سانسو
4937,209,دانست 4 مجم / 2 مل 3 امبول,دانست 4 مجم 3 امبول,82.5,دانست مجم امبول
65621,1534,رومانتيجرا 20 مجم 4 قرص,رومانتيجرا فياجرا,37.5,رومانتيجرا فياجرا
28570,206,ايراستابكس تريو 10/40/25 مجم 30 قرص,ايراستابكس تريو 104025 مجم 30 قرص جديد,123.0,ايراستابكس تريو 100 125 مجم 300 قرص جديد
24718,490,فاكتو مرهم 30 جم,فاكتو مرهم جديد,76.0,فاكتو كريم جديد
45062,87,دوروفين 30 كبسولة,دوروفين 30 كبسولة,153.0,دوروفين 300 كبسول


In [24]:
# search for @ in the seller_item_name
dataset_df[dataset_df['seller_item_name'].str.contains(".")]

,sku,marketplace_product_name_ar,seller_item_name,price,seller_item_name_clean
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 cap,56.5,estohalt 40 mg 14 cap
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
...,...,...,...,...,...
83557,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مم ق,39.0,سبانيلا 35 مم ق
83558,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مل اقراص,39.0,سبانيلا 35 مل اقراص
83559,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم مر 30 قرص,39.0,سبانيلا 35 مجم مر 30 قرص
83560,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم ام ار اقراص,39.0,سبانيلا 35 مجم ام ار اقراص


In [25]:
dataset_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price,seller_item_name_clean
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 cap,56.5,estohalt 40 mg 14 cap
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5,استوهالت 40 مجم 14 ك


In [26]:
X = dataset_df['seller_item_name']
y = dataset_df['sku']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [28]:
len(X_train), len(X_test)

(66849, 16713)